# IRENE-tests

Draft of tests for Irene-related functions

This notebook describes the city of IRENE, which carries the so-called preprocessing of the data, leading to PMAPS.

authors: J.J. Gomez-Cadenas


In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.core.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf

In [ ]:
from invisible_cities.core.core_functions import define_window

## Preproc Steps

**Irene takes care of the following chores**

1. Deconvolute RWF
2. Compute calibrated sum of PMTs (including MAU threshold)
3. Zero Suppression (ZS) in PMTs
4. S1 search (analysis specific)
5. S2 search (analysis specific) and rebin
6. ZS in SiPMs
7. Select S2 in ZS SiPMs 

For fast pre-proc, all the above steps must be computed on the fly (e.g, minimal access to disk) and
using pre-compiled (cython) functions in calculation-intensive parts (loops)

### Input files 

Required files (in $ICDIR/database/test_data)
1. **electrons_40keV_z250_RWF.h5** 

### Output files

Will be writen in temporary dir

1. **electrons_40keV_z250_PMAP.h5** 


## Case 1: electrons of 40 keV

In [ ]:
RWF_file = os.environ['ICDIR']  + '/database/test_data/electrons_40keV_z250_RWF.h5'
PMAP_file = os.environ['IC_DATA']  + '/electrons_40keV_z250_PMAP_TMP.h5'

In [ ]:
h5rwf = tb.open_file(RWF_file,'r+')

### pmtrwf and sipmrwf vectors

To get vectors use **get_vectors(file)** in tbl_functions

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT()
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

### Step 1: Deconvolution (from RWF to CWF)

In [ ]:
def cwf_from_rwf(pmtrwf, event_list, run_number=0, n_baseline=28000, thr_trigger=5):
    """Compute CWF from RWF"""
    DataPMT = load_db.DataPMT(run_number)
    coeff_c = DataPMT.coeff_c.values.astype(np.double)
    coeff_blr = DataPMT.coeff_blr.values.astype(np.double)
    
    CWF=[]
    for event in event_list:
        CWF.append(blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr,
                             n_baseline=n_baseline, 
                             thr_trigger=thr_trigger))
    return CWF
    

In [ ]:
CWF = cwf_from_rwf(pmtrwf, range(NEVT))

In [ ]:
wfm.plot_wfa_wfb(pmtblr[0], CWF[0], zoom=True, window_size=200)

In [ ]:
def compare_cwf_blr(cwf, pmtblr, event_list, window_size=500):
    """ Compute CWF  deconvoluting input RWF and compare with BLR."""


    DIFF = []
    for event in event_list:
        CWF = cwf[event]
        BLR = pmtblr[event]

        for i in range(len(BLR)):
            t0, t1 = define_window(BLR[i], window_size)

            diff = abs(np.sum(BLR[i][t0:t1]) - np.sum(CWF[i][t0:t1]))
            diff = 100. * diff/np.sum(BLR)
            DIFF.append(diff)
    
    return np.array(DIFF)

In [ ]:
def compare_cwf_blr_test():
    """Compare BLR and CWF"""
    RWF_file = os.environ['ICDIR']  + '/database/test_data/electrons_40keV_z250_RWF.h5'
    PMAP_file = os.environ['IC_DATA']  + '/electrons_40keV_z250_PMAP_TMP.h5'
    h5rwf = tb.open_file(RWF_file,'r+')
    pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)
    NEVT, NPMT, PMTWL = pmtrwf.shape
    
    CWF = cwf_from_rwf(pmtrwf, event_list=range(NEVT))
    diff = compare_cwf_blr(CWF, pmtblr, event_list=range(NEVT), window_size=200)
    
    for d in diff:
        assert d < 0.1
    
    return diff
    

In [ ]:
diff = compare_cwf_blr_test()

In [ ]:
    mpl.histo(diff, nbins=10, 
          title="diff BLR-CWF", xlabel="abs(e[blr] - e[cwf])", ylabel="Frequency")

### Step 2: calibrated PMT sum

#### The PMTs of the energy plane are calibrated (dividing by calibration constants) and added. 

In [ ]:
event=0
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
tstep = 25
signal_t = np.arange(0., PMTWL * tstep, tstep)

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=50, signal_end=150, 
                ymax = 150, 
                t_units='mus', units="pes")

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=99, signal_end=101, 
                ymax = 3, t_units='mus', units="pes")

S1 is very weak. A cut on 1.5 pes would completely kill the signal. 

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=120, signal_end=130, 
                ymax = 150, 
                t_units='mus', units="pes")

S2 shows the effect of the EL grid. The width of S2 is of the order of 3 mus (width of the grid). 

### Step 3: Zero suppression

A cut on 0.5 pes is set in the calibrated sum to maximize S1 efficiency

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum, ZS",
                signal_start=120, signal_end=130, 
                ymax = 150, 
                t_units='mus', units="pes")

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=99, signal_end=101, 
                ymax = 3, t_units='mus', units="pes")

### Step 4: Find S12 

**S1 and S2 signals are found from the ZS waveform**

1. S1: Search the first 600 mus (in the MC S1 is always at 100 mus but not in the data). The S1 signal must be at least 100 ns long (this is the minimum length of S1 after the shaping of the electronics) and not larger than 500 ns. The stride counts the number of bins that need to have signal. A stride of 4 means that there must be at least 0.5 pes each 100 ns (4 x 25 ns, where 25 ns is the DAQ sampling time). 

2. Search in the whole window (in the MC data, S2 can be anywhere after 100 mus, in the data it will tipically be placed in the middle of the DAQ window, thus one would search in the range [600, 1200] ns. The minimum length is 120 bins (120 x 25 = 3000 ns), no limit in the max length of the signal is not restricted (in order to avoid a bias, but one could further select the signal with a max width cut), the stride is taken to 40 (1 mus) and the signal is rebinned also in bins of 1 mus

In [ ]:
event = 0
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=4, lmax=20, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=100, lmax=1000000, stride=40,
                 rebin=True, rebin_stride=40)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=99, signal_end=101, 
                ymax = 3, t_units='mus', units="pes")

#### S1 and and S2 signals

In [ ]:
S1

#### S1 is empty (signal too weak for this event)

In [ ]:
S2

There is one S2 in this event. The format of S2 is:

**{s2_number:[array(T), array(E)]}**

where the ZS waveform is expressed by the list **[array(T), array(E)]**

#### Structure of S1/S2
1. Dictionary index counts the number of peaks
2. Dict values are a list [T,E], where T and E are numpy arrays of Time and Energy

#### Run the whole chain for the next event in the file (event 1)

In [ ]:
event = 1
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.25*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=8, lmax=20, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1100*units.mus, 
                 lmin=100, lmax=1000000, stride=40,
                 rebin=True, rebin_stride=40)
t1 = time()        
dt = t1 - t0

print("run  in {} s".format(dt))

In [ ]:
mpl.plot_signal(wfzs_t/units.mus, wfzs_ene, title="calibrated sum",
                signal_start=99, signal_end=101, 
                ymax = 3, t_units='mus', units="pes")

In [ ]:
S1

In [ ]:
plt.plot(S1[0][0],S1[0][1])

Number of photoelectrons in S1

In [ ]:
np.sum(S1[0][1])

In [ ]:
plt.plot(S2[0][0],S2[0][1])

Number of photoelectrons in S2

In [ ]:
np.sum(S2[0][1])

Width

In [ ]:
(S2[0][0][-1] - S2[0][0][0]) / units.mus

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

### SiPMs

In the case of MC Many of the SiPMs will contain exact zeros, since they had been ZS already at DIOMIRA level. For example, if we plot the first 16 SiPMs, we can see they are all exactly zero.

Find SiPMs with signal: the threshold is set to 1 adc count to simply get rid of all SiPMs which have exactly zero signal (the output of DIOMIRA is already ZS)

In [ ]:
event=1
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=1*units.adc) 

In [ ]:
sipm_i

In [ ]:
sf.plot_sipm_list(sipmrwf[event], sipm_i)

### Subtract baseline, and set a cut to supress dark current

in the MC the data comes already ZS. In data one needs to set a cut in order to ZS the SiPMs. 

In [ ]:
event=1
t0 = time()
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=25*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
t1 = time()        
dt = t1 - t0

print("run in {} s".format(dt))
print('number of SiPM with signal = {}'.format(len(SIPM)))

### PMAPS

#### Structure of a PMAP
1. S1 and S2 are dictionaries: each dictionary index is one S1/S2 candidate. Each dictionary value is a list which contains two elements. Element [0] is a np vector of times, element[1] is a np vector of energies in pes.
2. S2Si is a dictionary. Each index correspond to the S2 index. Each value is a list, whose length is equal to the number of SiPM with no zero signal in the S2 window. The list has as a first element the SiPM index and as a second the energy of each SiPM in the S2 window. Time is not neeeded (comes with S2).

#### S2 window
1. Given an S2 (T,E), obtain the relevant index range. 
2. Given a vector with SIPMs (energies above threshold), returns
    a list of np arrays. Each element of the list is the S2 window 
    in the SiPM (if not zero)
3. Given a vector with SIPMs (energies above threshold), and a 
    dictionary of S2s, S2d, returns a dictionary of SiPMs-S2.
    Each index of the dictionary correspond to one S2 and is
    a list of np arrays. Each element of the list is the S2 window 
    in the SiPM (if not zero)

## The full pre-proc 

In [ ]:
event = 1
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=0.5*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=6, lmax=20, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1190*units.mus, 
                 lmin=100, lmax=1000000, stride=40,
                 rebin=True, rebin_stride=40)
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=25*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=25*units.pes)
t1 = time()        
dt = t1 - t0
    
print("run in {} s".format(dt))

#### Plot SiPMs

In [ ]:
def scan_SIPM(SIPM):
    for i in range(4):
        for j in range(1):
            print('SIPM[{}][{}] = {}'.format(i,j, SIPM[i][j]))

In [ ]:
scan_SIPM(SIPM)

In [ ]:
sipm_i = sf.sipm_with_signal(sipmrwf[event], thr=1*units.adc) 

In [ ]:
sipm_i

In [ ]:
sf.plot_sensor_list_ene_map(sipmrwf[event], sipm_i, stype='SiPM')

In [ ]:
S2

In [ ]:
S2Si

In [ ]:
pf.plot_s2si_map(S2Si)

## Case 2: electrons of 511 keV

In [ ]:
RWF_file = os.environ['IC_DATA']  + '/electrons_511keV_z250_RWF.h5'
PMAP_file = os.environ['IC_DATA']  + '/electrons_511keV_z250_PMAP.h5'

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r+')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
event = 0
BLR = pmtblr[event]
RWF = pmtrwf[event]

In [ ]:
event=0
t0 = time()
CWF = blr.deconv_pmt(RWF, coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
t1 = time()        
dt = t1 - t0

print("run over  one event  in {} s".format(dt))

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=0, signal_end=400, 
                ymax = 350, 
                t_units='mus', units="pes")

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=99.5, signal_end=100.5, 
                ymax = 30, 
                t_units='mus', units="pes")

S1 is much stronger already. A cut on 1 pes and a width of 200 ns will be effective.

In [ ]:
mpl.plot_signal(signal_t/units.mus, csum, title="calibrated sum",
                signal_start=300, signal_end=400, 
                ymax = 350, 
                t_units='mus', units="pes")

**Parameters for 511 keV electrons**

1. Cut on CSUM raised to 1 pes.
2. Width of S1 raised to 8 samples (200 ns)

In [ ]:
t0 = time()
CWF = blr.deconv_pmt(pmtrwf[event], coeff_c, coeff_blr)
csum = cpf.calibrated_pmt_sum(CWF, adc_to_pes, n_MAU=100, thr_MAU=3) 
wfzs_ene, wfzs_indx = cpf.wfzs(csum, threshold=1.0*units.pes)
wfzs_t = cpf.time_from_index(wfzs_indx)
S1 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0,  tmax=590*units.mus, 
                 lmin=8, lmax=20, stride=4,
                 rebin=False)
S2 = cpf.find_S12(wfzs_ene, wfzs_indx, tmin=0*units.mus,  tmax=1190*units.mus, 
                 lmin=100, lmax=1000000, stride=40,
                 rebin=True, rebin_stride=40)
sipm = cpf.signal_sipm(sipmrwf[event], adc_to_pes_sipm, thr=25*units.pes, n_MAU=100)
SIPM = cpf.select_sipm(sipm)
S2Si = pf.sipm_s2_dict(SIPM, S2, thr=25*units.pes)
t1 = time()        
dt = t1 - t0
    
print("run in {} s".format(dt))

In [ ]:
pf.scan_s12(S1)

In [ ]:
pf.scan_s12(S2)

In [ ]:
pf.plot_s2si_map(S2Si)

In [ ]:
pf.scan_s2si_map(S2Si)

## Case 3: electrons of 1250 keV